<a href="https://colab.research.google.com/github/madh2802/CROP-DISEASE-PREDICTION-USING-TRANSFER-LEARNING/blob/main/CropDiseasePredictionUsingVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing necessary libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

## Importing google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Define input shape and number of classes

In [3]:
input_shape = (224, 224, 3)
num_classes = 38

## Load pre-trained VGG16 model and exclude top layers

In [4]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

58889256/58889256 [==============================] - 0s 0us/step


## Add custom top layers for our classification task

In [5]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)


## Define the new model

In [6]:
model = Model(inputs=base_model.input, outputs=x)

## Freeze some of the pre-trained layers

In [7]:
for layer in base_model.layers[:-8]:
    layer.trainable = False

## Compile the model

In [8]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


## Data augmentation for training set

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


## Data augmentation for validation and test sets

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)

## Load the data

In [11]:
train_dir = '/content/drive/MyDrive/data_distribution_for_SVM/train'
test_dir = '/content/drive/MyDrive/data_distribution_for_SVM/test'


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical')


Found 8751 images belonging to 38 classes.
Found 10547 images belonging to 38 classes.


##Train the model

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=8,  # Increase the number of epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)



Epoch 1/8
273/273 [==============================] - 6681s 24s/step - loss: 2.4346 - accuracy: 0.3666 - val_loss: 1.4416 - val_accuracy: 0.6076
Epoch 2/8
273/273 [==============================] - 189s 693ms/step - loss: 1.3741 - accuracy: 0.6169 - val_loss: 0.7795 - val_accuracy: 0.7682
Epoch 3/8
273/273 [==============================] - 223s 817ms/step - loss: 0.8778 - accuracy: 0.7444 - val_loss: 0.5369 - val_accuracy: 0.8435
Epoch 4/8
273/273 [==============================] - 187s 685ms/step - loss: 0.6454 - accuracy: 0.8104 - val_loss: 0.3896 - val_accuracy: 0.8890
Epoch 5/8
273/273 [==============================] - 187s 684ms/step - loss: 0.4971 - accuracy: 0.8548 - val_loss: 0.3782 - val_accuracy: 0.8876
Epoch 6/8
273/273 [==============================] - 226s 830ms/step - loss: 0.4149 - accuracy: 0.8765 - val_loss: 0.2665 - val_accuracy: 0.9193
Epoch 7/8
273/273 [==============================] - 227s 832ms/step - loss: 0.3141 - accuracy: 0.9046 - val_loss: 0.2448 - val_acc

##Save trained model

In [13]:
model.save('/content/drive/MyDrive/Saved_models/CropDiseasePredictionUsingVGG16.h5')

## Evaluate the model on the test set

In [14]:

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 10547 images belonging to 38 classes.


##Model accuracy


In [15]:
score = model.evaluate(test_generator, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.25772422552108765
Test accuracy: 0.9263297915458679


##Testing


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Trained_model.h5')

In [18]:
from PIL import Image
import numpy as np
# load image
image = Image.open('/content/013100cc-af7f-47eb-aaef-d46215549bc6 (1).JPG')

# resize image to (224, 224)
image = image.resize((224, 224))

# convert image to numpy array
image_array = np.array(image)

# add a batch dimension to the image
image_array = np.expand_dims(image_array, axis=0)

# make predictions
predictions = model.predict(image_array)

# get the index of the highest probability
predicted_class = np.argmax(predictions)


FileNotFoundError: ignored

In [ ]:
class_names = {
    0: 'Apple Scab',
    1: 'Apple Black Rot',
    2: 'Apple Cedar Rust',
    3: 'Apple healthy',
    4: 'Blueberry healthy',
    5: 'Cherry healthy',
    6: 'Cherry Powdery Mildew',
    7: 'Corn Gray Leaf Spot',
    8: 'Corn Common Rust',
    9: 'Corn healthy',
    10: 'Corn Northern Leaf Blight',
    11: 'Grape Black Rot',
    12: 'Grape Black Measles',
    13: 'Grape Leaf Blight',
    14: 'Grape healthy',
    15: 'Orange Huanglongbing',
    16: 'Peach Bacterial Spot',
    17: 'Peach healthy',
    18: 'Bell Pepper Bacterial Spot',
    19: 'Bell Pepper healthy',
    20: 'Potato Early Blight',
    21: 'Potato healthy',
    22: 'Potato Late Blight',
    23: 'Raspberry healthy',
    24: 'Soybean healthy',
    25: 'Squash Powdery Mildew',
    26: 'Strawberry Healthy',
    27: 'Strawberry Leaf Scorch',
    28: 'Tomato Bacterial Spot',
    29: 'Tomato Early Blight',
    30: 'Tomato Late Blight',
    31: 'Tomato Leaf Mold',
    32: 'Tomato Septoria Leaf Spot',
    33: 'Tomato Two Spotted Spider Mite',
    34: 'Tomato Target Spot',
     35: 'Tomato Mosaic Virus',
    36: 'Tomato Yellow Leaf Curl Virus',
    37: 'Tomato healthy'
}


In [ ]:
class_name = class_names[predicted_class]
print(class_name)
